# Tutorial 3: How simulations define your predictions
The inverse problem has no unique solution as it is ill-posed. In order to solve it we need to constraint the space of possible solutions. While inverse solutions like minimum-norm estimates have an explicit constraint of minimum-energy, the constraints with esinet are implicit and mostly shaped by the simulations.

This tutorial aims the relation between simulation parameters and predictions.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
plot_params = dict(surface='white', hemi='both', verbose=0)


## Create Forward model
First we create a template forward model which comes with the esinet package

In [2]:
info = get_info()
fwd = create_forward_model(info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    1.2s remaining:    2.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    1.2s remaining:    0.7s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.1s finished


## Simulate
Next, we simulate two types of data: 
1. Data containing small sources with 15-25 mm in diameter.
2. Data containing large sources with 35-45 mm in diameter.

In [3]:
n_samples = 10000
settings_small = dict(n_sources=(1, 10), extents=(15, 25))
settings_large = dict(n_sources=(1, 10), extents=(35, 45))

sim_small = Simulation(fwd, info, settings=settings_small).simulate(n_samples=n_samples)
sim_large = Simulation(fwd, info, settings=settings_large).simulate(n_samples=n_samples)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

## Lets visualize the two types of simulations
The two brain plots should now look quite different, as one contains large and extended sources while the other contains tiny point-like sources.

In [4]:
brain = sim_small.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small sources', 'title',
               font_size=14)

brain = sim_large.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large sources', 'title',
               font_size=14)

## Train individual neural networks

In [5]:
net_small = Net(fwd, verbose=True).fit(sim_small)
net_large = Net(fwd, verbose=True).fit(sim_large)

Model: "net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 1284)              165636    
Total params: 173,572
Trainable params: 173,572
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
71/71 [==============================] - 1s 6ms/step - loss: 0.0018 - mean_squared_error: 0.0038 - val_loss: 0.0016 - val_mean_squared_error: 0.0037
Epoch 2/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0016 - mean_squared_error: 0.0037 - val_loss: 0.0016 - val_mean_squared_error: 0.0037
Epoch 3/100
71/71 [==============================] - 0s 5ms/step - loss: 0.0016 - mean_squared_error: 0.0036 - val_loss: 0.0016 - val_mean_squared_error: 0.0036
Epoch 4/100
71/71 [=================

Now we have simulated two different types of source & eeg data and build two neural networks that each was trained on one of these simulations. Lets see how they perform within their own simulation type.

In [6]:
# Simulate some new, unseen test data    
n_test_samples = 1
sim_test_small = Simulation(fwd, info, settings=settings_small).simulate(n_samples=n_test_samples)
sim_test_large = Simulation(fwd, info, settings=settings_large).simulate(n_samples=n_test_samples)


brain = sim_test_small.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of small data', 'title',
               font_size=14)


brain = net_small.predict(sim_test_small).plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small-Net on small data', 'title',
               font_size=14)



brain = sim_test_large.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of large data', 'title',
               font_size=14)


brain = net_large.predict(sim_test_large).plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large-Net on large data', 'title',
               font_size=14)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

(1, 61, 1)
(1, 61, 1)


Now we will use the large-net to predict the small simulation and vice versa.

In [7]:
brain = sim_test_small.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of small data', 'title',
               font_size=14)


brain = net_large.predict(sim_test_small).plot(**plot_params)
brain.add_text(0.1, 0.9, 'Large-Net on small data', 'title',
               font_size=14)



brain = sim_test_large.source_data.plot(**plot_params)
brain.add_text(0.1, 0.9, 'Ground Truth of large data', 'title',
               font_size=14)


brain = net_small.predict(sim_test_large).plot(**plot_params)
brain.add_text(0.1, 0.9, 'Small-Net on large data', 'title',
               font_size=14)

(1, 61, 1)
(1, 61, 1)


We now find that the Net which was trained on large simulations always tends to find large sources - even when confronted with data in which small sources were active. 

Conversely, the Net which was trained on simulations that contain small sources finds sparse sources when confronted with data containing large-source activity.

This demonstrates that our simulation settings function like priors. Further, it emphasizes the importance to state your priors and to motivate your choice.

In many cases we can't make a choice and we want to make as few assumptions into our models as possible. In this case we propose that you use large ranges in your settings to maximize the diversity of your training data.

A sample of a diverse setting is given in the next cell:

In [ ]:
settings = {
    'number_of_sources': (1, 20),  # The range of simultaneously active sources.
    'extents': (2, 50),  # The range of source diameters in mm 
    'amplitudes': (1, 100),  # Defines the range of amplitudes (in arbitrary units)
    'shapes': 'both',  # Gives you both gaussian-shaped and flat sources
    'beta': (0, 1.5),  # Defines the distribution of the noise in terms of 1/f**beta
}